In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
train=pd.read_csv("https://raw.githubusercontent.com/ezioauditore-tech/AI/main/datasets/Titanic/train.csv")
test=pd.read_csv("https://raw.githubusercontent.com/ezioauditore-tech/AI/main/datasets/Titanic/test.csv")

In [3]:
train.info(),test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Pclass  

(None, None)

In [4]:
train.set_index("PassengerId",inplace=True)
test.set_index("PassengerId",inplace=True)

In [5]:
train

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...
887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S


In [6]:
y_train=train["Survived"]

In [7]:
train.drop(labels="Survived",axis=1,inplace=True)

In [8]:
train.shape

(891, 10)

In [9]:
train_test=train.append(test)

<ipython-input-9-d2734cde2921>:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  train_test=train.append(test)


In [10]:
#delete columns that are not used as features for training and prediction
columns_to_drop=["Name","Age","SibSp","Ticket","Cabin","Parch","Embarked"]
train_test.drop(labels=columns_to_drop,axis=1,inplace=True)

In [11]:
train_test

,Pclass,Sex,Fare
PassengerId,,,
1,3,male,7.2500
2,1,female,71.2833
3,3,female,7.9250
4,1,female,53.1000
5,3,male,8.0500
...,...,...,...
1305,3,male,8.0500
1306,1,female,108.9000
1307,3,male,7.2500


In [12]:
#cover objects to numbers by pandas.grt_dummies
train_test_dummies=pd.get_dummies(train_test,columns=["Sex"])

In [13]:
train_test_dummies.isnull().sum()

Pclass        0
Fare          1
Sex_female    0
Sex_male      0
dtype: int64

In [14]:
train_test_dummies.fillna(value=0.0,inplace=True)

In [15]:
X_train=train_test_dummies.values[0:891]
X_test=train_test_dummies.values[891:]

In [16]:
from sklearn.preprocessing import MinMaxScaler
scaler=MinMaxScaler()
X_train_scale=scaler.fit_transform(X_train)
X_test_scale=scaler.transform(X_test)

In [17]:
#split training feature and target sets into training and validation subsets(to find y value because kaggle dataset doesn't have dependent variable y)
from sklearn.model_selection import train_test_split
X_train_sub, X_validation_sub, y_train_sub, y_validation_sub=train_test_split(X_train_scale,y_train,random_state=0)

In [18]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import classification_report

In [19]:
learning_rates=[0.05,0.1,0.25,0.5,0.75,1]
for learning_rate in learning_rates:
  gb=GradientBoostingClassifier(n_estimators=20,learning_rate=learning_rate,max_features=2,max_depth=2,random_state=0)
  gb.fit(X_train_sub,y_train_sub)
  print("Leraning rate:",learning_rate)
  print("Accuracy score (training): {0:.3f}".format(gb.score(X_train_sub,y_train_sub)))
  print("Accuracy score (validation): {0:.3f}".format(gb.score(X_validation_sub,y_validation_sub)))
  print()

Leraning rate: 0.05
Accuracy score (training): 0.789
Accuracy score (validation): 0.780

Leraning rate: 0.1
Accuracy score (training): 0.792
Accuracy score (validation): 0.780

Leraning rate: 0.25
Accuracy score (training): 0.816
Accuracy score (validation): 0.803

Leraning rate: 0.5
Accuracy score (training): 0.826
Accuracy score (validation): 0.834

Leraning rate: 0.75
Accuracy score (training): 0.831
Accuracy score (validation): 0.789

Leraning rate: 1
Accuracy score (training): 0.831
Accuracy score (validation): 0.789



In [20]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import classification_report

In [21]:
gb1=AdaBoostClassifier(n_estimators=50,random_state=0)
gb1.fit(X_train_sub,y_train_sub)
print("Accuracy score (training): {0:.3f}".format(gb1.score(X_train_sub,y_train_sub)))
print("Accuracy score (validation): {0:.3f}".format(gb1.score(X_validation_sub,y_validation_sub)))
print()

Accuracy score (training): 0.802
Accuracy score (validation): 0.776



In [ ]:
learning_rates=[0.05,0.1,0.25,0.5,0.75,1]
for learning_rate in learning_rates:
  gb2=AdaBoostClassifier(n_estimators=7000000,learning_rate=learning_rate,random_state=0)
  gb2.fit(X_train_sub,y_train_sub)
  print("Leraning rate:",learning_rate)
  print("Accuracy score (training): {0:.3f}".format(gb2.score(X_train_sub,y_train_sub)))
  print("Accuracy score (validation): {0:.3f}".format(gb2.score(X_validation_sub,y_validation_sub)))
  print()